In [1]:
# Download CBDB sqlite
# !git clone https://github.com/cbdb-project/cbdb_sqlite.git
import requests
import os
# url = "https://hf-mirror.com/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url, verify=False)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

d:\anaconda\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hf-mirror.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\anaconda\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdn-lfs-us-1.hf-mirror.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# import glob
import os

latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'latest.db'

In [4]:
# # Explode latest database file
# !pip install pyunpack
# !pip install patool
# from pyunpack import Archive
# Archive(latest_file).extractall(".")

import py7zr

# 指定压缩文件和目标目录
archive = './cbdb_sqlite/latest.7z'
output_dir = ''

# 解压文件
with py7zr.SevenZipFile(archive, 'r') as archive:
    archive.extractall(output_dir)

print("解压完成！")



解压完成！


In [5]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [6]:
sql_str = "SELECT bm.c_dy, d.c_dynasty_chn, count(distinct m.c_textid) as count\
        FROM biog_main bm\
        JOIN (\
            SELECT bs.c_personid, bs.c_textid\
            FROM biog_source_data bs\
        ) m ON bm.c_personid = m.c_personid\
        JOIN dynasties d ON bm.c_dy = d.c_dy\
        GROUP BY bm.c_dy;"
dycount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
dycount_df

,c_dy,c_dynasty_chn,count
0,0,未詳,117
1,1,漢前,5
2,2,秦漢,18
3,3,三國,3
4,4,南北朝,9
...,...,...,...
69,76,西燕,2
70,77,周,13
71,78,西夏,3
72,81,鄭（王世充）,1


In [7]:
sql_str="select bm.c_dy, d.c_dynasty, d.c_dynasty_chn, group_concat(bs.c_textid)as bookids\
        from biog_main bm\
        join biog_source_data bs on bm.c_personid = bs.c_personid\
        join dynasties d on bm.c_dy = d.c_dy\
        group by bm.c_dy;"
dycount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
dycount_df['bookids'] = dycount_df['bookids'].apply(lambda x: ','.join(set(x.split(','))) if isinstance(x, str) else x)  
dycount_df['count'] = dycount_df['bookids'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)  
dycount_df

,c_dy,c_dynasty,c_dynasty_chn,bookids,count
0,0,unknown,未詳,"4626,62087,18417,65079,38457,4591,27157,14561,...",117
1,1,Pre-Han,漢前,"66734,9601,5908,6134,7596",5
2,2,QinHan,秦漢,"3291,38772,3689,38733,66734,2776,38742,2493,96...",18
3,3,SanGuo,三國,"9602,66734,9601",3
4,4,NanBei Chao,南北朝,"66734,2067,9601,32033,5906,32039,32038,38295,4801",9
...,...,...,...,...,...
69,76,Western Yan (16 states),西燕,"66734,24309",2
70,77,Zhou (Wu Zetian),周,"6267,3689,66734,40303,24309,38296,4578,32033,3...",13
71,78,Western Xia,西夏,"17138,7596,66734",3
72,81,Zheng (Wang Shichong),鄭（王世充）,32038,1


In [8]:
dycount_df_filtered = dycount_df[dycount_df["c_dy"].isin([6, 15, 16, 17, 18, 19 ,20])]  
dycount_df_filtered

,c_dy,c_dynasty,c_dynasty_chn,bookids,count
6,6,Tang,唐,"4626,4394,24670,16881,38457,62087,5774,38863,8...",124
15,15,Song,宋,"3358,40312,4591,56237,27157,2308,2224,4707,250...",384
16,16,Liao,遼,"6267,66734,40303,24309,9601,32033,17138,8947,7596",9
17,17,Jin,金,"38333,5725,38533,4626,40334,9601,38397,4802,38...",41
18,18,Yuan,元,"21611,4626,4418,32058,38300,38302,38397,62087,...",162
19,19,Ming,明,"58118,13311,32049,4591,6931,4707,27143,46317,2...",499
20,20,Qing,清,"6329,4626,38578,20177,27046,38397,27925,62087,...",222


In [9]:
dycount_df_filtered.to_csv('text_statistic.csv', index=False)

In [11]:
# Read the content of old JavaScript files
with open('script.js', 'r',encoding='utf-8') as file:  
    js_code = file.read()  
  
# Delete old data  
start_delimiter = 'const chart3Data = ['  
end_delimiter = '];// Chart 3 Data'
start_index = js_code.find(start_delimiter)  
end_index = js_code.find(end_delimiter) + len(end_delimiter)  
js_code = js_code[:start_index] + js_code[end_index:]
# new data
js_data = ',\n'.join(    
    f'{{ label: "{row["c_dynasty"]}", value: {row["count"]}, id: "{row["c_dynasty"]}"}}'    
    for index, row in dycount_df_filtered.iterrows()    
)  
new_data = f'{start_delimiter}\n{js_data}\n{end_delimiter}'  
  
js_code = new_data +'\n' + js_code  
  
# Write updated JavaScript file 
with open('script.js', 'w',encoding='utf-8') as file:  
    file.write(js_code)   

In [13]:
# update the date
import datetime  
import re  

svg_path = os.path.join(os.getcwd(), 'images', 'svg6.svg')  
with open(svg_path, 'r',encoding='utf-8') as file:  
    svg_content = file.read()  
  
# get current date  
current_time = datetime.datetime.now().strftime('%Y年%m月%d日') 
current_time_english = datetime.datetime.now().strftime('%B %d, %Y') 
  
updated_svg_content = re.sub(r'更新日期：\d+年\d+月\d+日', f'更新日期：{current_time}', svg_content)    
updated_svg_content = re.sub(r'Updated on: \w+ \d+, \d+', f'Updated on: {current_time_english}', updated_svg_content)
with open(svg_path, 'w',encoding='utf-8') as file:  
    file.write(updated_svg_content) 

In [14]:
# close the db
CONN.close()